# TF-IDF

In [26]:
import pandas as pd
import numpy as np

from pandas.errors import EmptyDataError

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [27]:
date_str = '20250822'

In [28]:
def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except EmptyDataError:
        print(f"⚠️ {path} 는 비어있음 → 스킵")
        return pd.DataFrame()   # 빈 DF 반환
    except FileNotFoundError:
        print(f"❌ {path} 파일 없음 → 스킵")
        return pd.DataFrame()

# 파일 불러오기
raw_data_i = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{date_str}.csv")
raw_data_g = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{date_str}.csv")
raw_data_gen_1 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_00000-19999.csv")
raw_data_gen_2 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_20000-39999.csv")
raw_data_gen_3 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_40000-59999.csv")
raw_data_gen_4 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_60000-79999.csv")
raw_data_gen_5 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_80000-99999.csv")

# 하나의 파일로 유니온 (비어있으면 자동 무시)
raw_data = pd.concat([
    raw_data_gen_1, raw_data_gen_2, raw_data_gen_3,
    raw_data_gen_4, raw_data_gen_5,
    raw_data_i, raw_data_g
], ignore_index=True)

print("총 데이터 크기:", raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250822_00000-19999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250822_20000-39999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250822_40000-59999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250822_80000-99999.csv 는 비어있음 → 스킵
총 데이터 크기: (245, 4)


,url,title,text,publish_date,text_clean,title_clean
0,https://www.hankyung.com/article/2025082260104,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다 회담은 뉴욕 증시가 마감할 무렵 ...,한국경제
1,https://www.hankyung.com/article/2025082260177,"""그만해 달라"" 외침에도…또래 뺨 때린 '촉법' 중학생 송치",사진=연합뉴스\n\n폭행을 멈춰달라는 부탁에도 또래의 뺨을 계속해서 때린 중학생과 ...,2025-08-22 15:29:09+09:00,사진 연합뉴스 폭행을 멈춰달라는 부탁에도 또래의 뺨을 계속해서 때린 중학생과 옆에...,그만해 달라 외침에도 또래 뺨 때린 촉법 중학생 송치
11,https://www.hankyung.com/article/2025082260597,처자식 죽이고 자신만 살아남은 父의 선처 요구…판사 '호통',생활고 탓으로 두 아들과 아내 살해\n\n6월 2일 전남 진도군 임회면 진도항에서 ...,2025-08-22 15:42:11+09:00,생활고 탓으로 두 아들과 아내 살해 월 일 전남 진도군 임회면 진도항에서 지씨...,처자식 죽이고 자신만 살아남은 의 선처 요구 판사 호통


In [29]:
# 불용어제거 -> 형태소 조사 삭제 -> tf-idf -> 가장 중요한 10개 단어 뽑아내기

In [30]:
# 불용어 불러오기 (줄바꿈 기준 분리)
with open("/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 조사 제거 → 불용어 제거 순차 적용
df['title_clean'] = df['title_clean'].apply(remove_josa)  # 조사 제거
df['title_clean'] = df['title_clean'].apply(remove_stopwords)  # 불용어 제거

df['text_clean'] = df['text_clean'].apply(remove_josa)  # 조사 제거
df['text_clean'] = df['text_clean'].apply(remove_stopwords)  # 불용어 제거

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1) 코퍼스 준비 (NaN 방지)
corpus = df['text_clean'].fillna('')

# 2) 벡터라이저 설정
# - tokenizer=lambda s: s.split(): 이미 공백으로 토큰화된 텍스트를 그대로 사용
# - lowercase=False: 한글/대소문 유지
# - ngram_range=(1,1): 단어 단위(필요 시 (1,2) 등으로 확장)
# - min_df, max_df, max_features는 데이터 규모에 맞게 조절
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    token_pattern=None,
    lowercase=False,
    ngram_range=(1,1),
    min_df=2,           # 2개 미만 문서에만 나온 단어 제거
    max_df=0.7,         # 70% 넘는 문서에 등장하는 흔한 단어 제거
    max_features=8000   # 상위 8천개 단어만 사용 (메모리 절약용)
)

# 3) 학습 및 변환 (희소행렬)
X = vectorizer.fit_transform(corpus)   # shape: (문서수, 단어수)
terms = vectorizer.get_feature_names_out()

# 4) (옵션) DataFrame으로 시각화 — 메모리 주의
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=terms)

# 5) 각 문서별 TF-IDF 상위 키워드 뽑기 함수
def top_tfidf_terms(row_index: int, topk: int = 10):
    row = X.getrow(row_index).toarray().ravel()
    if not np.any(row):
        return []
    idx = np.argsort(row)[::-1][:topk]
    return list(zip(terms[idx], row[idx]))

# 6) 모든 문서에 대해 상위 키워드 뽑아 df에 붙이기 (리스트→문자열)
topk = 10
df['tfidf_top_terms'] = [
    ', '.join([f'{t}:{w:.3f}' for t, w in top_tfidf_terms(i, topk)]) 
    for i in range(X.shape[0])
]

In [32]:
df.head()

,url,title,text,publish_date,text_clean,title_clean,tfidf_top_terms
0,https://www.hankyung.com/article/2025082260104,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령 푸틴 러시아 대통령 만났습니다 회담 뉴욕 증시 마감 관망 나타났습니다...,한국,"관세:0.470, 물가:0.231, 금리:0.228, 인플레이션:0.205, 수입:..."
1,https://www.hankyung.com/article/2025082260177,"""그만해 달라"" 외침에도…또래 뺨 때린 '촉법' 중학생 송치",사진=연합뉴스\n\n폭행을 멈춰달라는 부탁에도 또래의 뺨을 계속해서 때린 중학생과 ...,2025-08-22 15:29:09+09:00,연합뉴스 폭행 멈춰 달라 부탁 또래 뺨 계속 때린 중학생 범행 부추긴 고등학생 가정...,그만해 달라 외침 또래 뺨 때린 촉법 중학생 송치,"영상:0.476, 경찰:0.293, C:0.254, 범행:0.220, SNS:0.2..."
11,https://www.hankyung.com/article/2025082260597,처자식 죽이고 자신만 살아남은 父의 선처 요구…판사 '호통',생활고 탓으로 두 아들과 아내 살해\n\n6월 2일 전남 진도군 임회면 진도항에서 ...,2025-08-22 15:42:11+09:00,생활고 아들 아내 살해 전남 진도군 임회면 진도항 지씨 아내 아들 태워 바다로 돌진...,처자 죽 살아남은 선처 요구 판사 호통,"아내:0.334, 아들:0.330, 제출:0.257, 검찰:0.217, 재판:0.2..."
12,https://www.hankyung.com/article/2025082260616,"코스피, 이틀 연속 오르며 3160선 회복…조선·방산·원전 '반등'",코스닥도 '상승'\n\n원·달러 환율 1400원 목전서 상승세 꺾여\n\n사진=뉴스...,2025-08-22 15:51:18+09:00,코스닥 환율 목전 상승세 꺾여 뉴스 제롬 파월 미국 중앙은행 Fed 연설 코스피 코...,코스피 이틀 연속 오르며 회복 조선 방산 원전 반등,"코스닥:0.398, 파월:0.230, 마감:0.204, 코스피:0.199, 지수:0..."
13,https://www.hankyung.com/article/2025082260667,"李대통령 ""사람 살리는 금융정책 강구…불법추심에 삶 꺾이면 안돼""","""금융취약계층 자살, 추심 등 원인""…제도 개선 주문\n\n전세사기 피해자 자살엔 ...",2025-08-22 15:42:50+09:00,금융 취약 계층 자살 추심 원인 제도 개선 주문 전세 사기 피해자 자살 전세 대출 ...,대통령 살리는 금융정책 강구 불법 추심 삶 꺾이면,"자살:0.401, 추심:0.378, 금융:0.359, 전세:0.298, 대통령:0...."


In [33]:
from collections import Counter
import re

# 1. 전체 키워드 모으기 (점수 제거)
all_terms = []
for row in df['tfidf_top_terms']:
    if not isinstance(row, str):
        continue
    # "단어:점수" → 단어만 추출
    terms = [re.split(r':', t)[0] for t in row.split(', ')]
    all_terms.extend(terms)

# 2. 빈도수 계산
term_freq = Counter(all_terms)

# 3. DataFrame으로 변환
term_freq_df = pd.DataFrame(term_freq.items(), columns=['term', 'count'])
term_freq_df = term_freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

term_freq_df.head(10)

,term,count
0,,33
1,AI,14
2,미국,14
3,기술,12
4,대통령,8
5,책,8
6,정부,8
7,트럼프,8
8,경찰,7
9,매출,7


In [34]:
term_freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/text_TFIDF.csv", index=False)